In [1]:
import pandas as pd
import psutil
import time

In [2]:
from dmagellan.blocker.blackbox.blackbox_blocker import BlackBoxBlocker
from dask import multiprocessing

In [44]:
A = pd.read_csv('../datasets/sample_citeseer_100k.csv')
B = pd.read_csv('../datasets/sample_dblp_100k_mur.csv')
A = A.head(5000)
B = B.head(5000)

In [34]:
import py_stringsimjoin

In [35]:
import py_stringmatching
from py_stringmatching.similarity_measure.jaccard import Jaccard

In [36]:
A.tail()

,id,title,authors,journal,month,year,publication_type
995,1636434,Federal Reserve Board,Arthur B. Kennickell,NaN,NaN,1996.0,NaN
996,89382,An analysis of the 1998 Workplace Employee Relations Survey,"S Dex, C Smith, Colin Smith",NaN,NaN,2001.0,NaN
997,1613860,Jackendoffs Semantic Structures,Manuel Bremer,NaN,NaN,NaN,NaN
998,912662,Power-aware Scheduling for Parallel Security Processors with Analytical Models,"Yung-chia Lin, Yi-ping You, Chung-wen Huang, Jenq-kuen Lee, Wei-kuan Shih, Ting-ting Hwang",NaN,NaN,NaN,NaN
999,314332,Disclaimer.........................................................................................,"John Harvey, Fenella Long",NaN,NaN,1999.0,NaN


In [42]:
def blacbox_fn(ltuple, rtuple):
    jac = Jaccard()
    t1, a1 = ltuple['title'], ltuple['authors']
    if pd.isnull(t1) or pd.isnull(a1):
        return True
    t2, a2 = rtuple['title'], rtuple['authors']
    if pd.isnull(t2) or pd.isnull(a2):
        return True
    if len(t1) > 150 and len(t2) > 150 and len(a1) > 50  and len(a2) > 50:
        return False
    
    
    return True

In [45]:
memUsageBefore = psutil.virtual_memory().used/1e9
timeBefore = time.time()
bb = BlackBoxBlocker()
bb.set_black_box_function(blacbox_fn)
bb.set_ltable_attrs(['title', 'authors'])
bb.set_rtable_attrs(['title', 'authors'])
C = bb.block_tables(A, B , 'id', 'id', nltable_chunks=2,
                    nrtable_chunks=2, compute=False)

D = C.compute(get=multiprocessing.get)

timeAfter = time.time()
memUsageAfter = psutil.virtual_memory().used/1e9
print(len(D))

2067
